In [1]:
# note during the execution of this file, change the n number and excute...

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2

from sklearn.feature_selection import RFE


In [2]:

def rfeFeature(indep,dep,n):
    logistic_model= LogisticRegression(solver="lbfgs")
    log_rfe= RFE(estimator=logistic_model,n_features_to_select=n)
    log_fit= log_rfe.fit(indep,dep)
    log_rfe_feature_Xnew =log_fit.transform(indep)

    selected_features= log_fit.get_support()
    
    return log_rfe_feature_Xnew, selected_features

def Split_Scaler(indep,dep):
    x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)
    scX=StandardScaler()
    x_train=scX.fit_transform(x_train)
    x_test=scX.transform(x_test)
    return x_train,x_test,y_train,y_test
    
    # save the splitscaler method
    pickle.dump(scX,open("scX.pkl","wb"))


def cm_predition(classifier, x_test):
    y_pred=classifier.predict(x_test)

    #making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(y_test,y_pred)

    # making Classification Report
    from sklearn.metrics import classification_report
    clf_report=classification_report(y_test,y_pred)

    # finding the accuracy score
    from sklearn.metrics import accuracy_score
    accuracy_score=accuracy_score(y_test,y_pred)
    
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred

def logistic(x_train,y_train,x_test):
    from sklearn.linear_model import LogisticRegression
    classifier=LogisticRegression(random_state=0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 
        

# Displaying the values in the tabular frame

def rfe_classification(acc_logistic):
    
    df=pd.DataFrame(index=["Logistic"],columns=["Logistic"])
    df['Logistic']= acc_logistic     
    return df


In [3]:
dataset=pd.read_csv("prep.csv",index_col=None)
df1=dataset  

#Convert categorical variable into dummy/indicator variables.
df1=pd.get_dummies(df1,drop_first=True,dtype=int)
df1

# separation of independent and dependent variables
indep=df1.drop("classification_yes",axis=1)
dep=df1["classification_yes"]
      
log_rfe_feature_Xnew, selected_features =rfeFeature(indep, dep, 5)

x_train,x_test,y_train,y_test = Split_Scaler(log_rfe_feature_Xnew, dep) 

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = logistic(x_train,y_train,x_test)

result= rfe_classification(accuracy_score)
result

# Save the model
pickle.dump(classifier,open("finalized_logistic_model","wb"))


/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

In [4]:
result #5

,Logistic
Logistic,0.975


In [5]:
# selected features

selected_features=indep.columns[selected_features]

print("selected features: ",selected_features)

selected features:  Index(['al', 'sg_c', 'sg_d', 'htn_yes', 'dm_yes'], dtype='object')


In [6]:
df1[selected_features]

,al,sg_c,sg_d,htn_yes,dm_yes
0,3.0,1,0,0,0
1,2.0,1,0,0,0
2,1.0,0,0,0,0
3,1.0,0,1,0,0
4,0.0,1,0,0,0
...,...,...,...,...,...
394,0.0,0,0,0,0
395,0.0,1,0,1,1
396,3.0,1,0,1,1
397,0.0,0,0,1,1
